In [9]:
import numpy as np
import torch
import torchvision
from torch import Tensor
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import os
import math
import time

In [10]:
def test_sep_separable( in_channels, out_channels, H, W, kernel_size,  stride, padding, numthreads):
    we_depth = torch.randint(-1, 2, (in_channels, 1, kernel_size, kernel_size), dtype=torch.float32)
    we_sep = torch.randint(-1, 2, (out_channels, in_channels, 1, 1), dtype=torch.float32)
    inp = torch.randint(-256, 257, (1, in_channels, W, H), dtype=torch.float32)
    #outp = torch.nn.functional.conv2d(inp, we_depth, None, stride, 1, 1, in_channels)
    #outp = torch.nn.functional.conv2d(outp, we_sep, None, stride, 0, 1, 1)
    outp = torch.nn.functional.conv2d(inp, we_sep, None, stride, 0, 1, 1)
    if out_channels > 1:
        outp = torch.squeeze(outp, 0)
    inp_numpy = inp.numpy()
    we_depth_numpy = we_depth.numpy()
    we_sep_numpy = we_sep.numpy()
    outp_numpy = outp.numpy()

    np.savetxt("inp.txt", inp_numpy.flatten(), "%d")
    np.savetxt("weight_depth.txt", we_depth_numpy.flatten(), "%d")
    np.savetxt("weight_sep.txt", we_sep_numpy.flatten(), "%d")
    np.savetxt("outp.txt", outp_numpy.flatten(), "%d")
    with open("params.txt", "w") as f:
        f.write(f'{numthreads} {out_channels} {in_channels} {H} {W} {kernel_size} {stride} {padding}')
    !gcc -march=native -O3 -g -o  blas_test blas_test.c -lblas -lm
    !gcc -march=native -O3 -g -o conv_sep_ex conv_sep_ex.c
    !time ./conv_sep_ex
    !time ./blas_test

In [11]:
test_sep_separable( 64, 64, 640, 640, 1,  1, 0, 1)

blas_test.c: In function ‘transpose’:
blas_test.c:51:45: warning: passing argument 5 of ‘schedule1d’ discards ‘volatile’ qualifier from pointer target type []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wdiscarded-qualifiers-Wdiscarded-qualifiers]8;;]
   51 |     schedule1d(0, M, thread_no, NUMTHREADS, &istart, &iend);
      |                                             ^~~~~~~
blas_test.c:16:78: note: expected ‘int *’ but argument is of type ‘volatile int *’
   16 | nt beg, int fin, int thread_no, int number_of_threads, int* op_start, int* op_end){
      |                                                        ~~~~~^~~~~~~~

blas_test.c:51:54: warning: passing argument 6 of ‘schedule1d’ discards ‘volatile’ qualifier from pointer target type []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wdiscarded-qualifiers-Wdiscarded-qualifiers]8;;]
   51 |     schedule1d(0, M, thread_no, NUMTHREADS, &istart, &iend);
      |                            

In [12]:
# !gcc -march=native -O3 -g -o  blas_test blas_test.c -lblas -lm
# !./blas_test